# Task 1: Exploratory Data Analysis (EDA) & Statistical Analysis

## Objective
Develop a foundational understanding of the insurance data, assess its quality, and uncover initial patterns in risk and profitability.

## Key Questions to Answer:
1. What is the overall Loss Ratio (TotalClaims / TotalPremium) for the portfolio? How does it vary by Province, VehicleType, and Gender?
2. What are the distributions of key financial variables? Are there outliers in TotalClaims or CustomValueEstimate?
3. Are there temporal trends? Did the claim frequency or severity change over the 18-month period?
4. Which vehicle makes/models are associated with the highest and lowest claim amounts?


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')
from utils import (
    load_data, calculate_loss_ratio, calculate_claim_frequency,
    calculate_claim_severity, calculate_margin
)
from data_processing import (
    load_and_validate_data, get_data_summary, check_missing_values,
    detect_outliers_iqr, calculate_descriptive_stats
)
from eda import (
    plot_numeric_distribution, plot_categorical_distribution,
    plot_boxplot, plot_correlation_matrix, plot_temporal_trends,
    plot_loss_ratio_by_category, create_insightful_visualization_1,
    create_insightful_visualization_2, create_insightful_visualization_3
)

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")


## 1. Data Loading and Initial Inspection


In [ ]:
# Load the data
# Note: Update the path to your actual data file location
data_path = '../data/raw/insurance_data.csv'  # Update this path

try:
    df = load_and_validate_data(data_path)
    print(f"Data loaded successfully!")
    print(f"Shape: {df.shape}")
except FileNotFoundError:
    print(f"Data file not found at {data_path}")
    print("Please update the data_path variable with the correct path to your insurance data.")
    print("\nCreating sample structure for demonstration...")
    # Create a sample dataframe structure for demonstration
    df = pd.DataFrame({
        'PolicyID': range(1000),
        'TransactionMonth': pd.date_range('2014-02-01', periods=1000, freq='D'),
        'Province': np.random.choice(['Gauteng', 'Western Cape', 'KwaZulu-Natal', 'Eastern Cape'], 1000),
        'PostalCode': np.random.randint(1000, 9999, 1000),
        'Gender': np.random.choice(['Male', 'Female'], 1000),
        'VehicleType': np.random.choice(['Sedan', 'SUV', 'Hatchback', 'Coupe'], 1000),
        'Make': np.random.choice(['Toyota', 'Ford', 'BMW', 'Mercedes'], 1000),
        'TotalPremium': np.random.uniform(5000, 50000, 1000),
        'TotalClaims': np.random.uniform(0, 60000, 1000),
        'CustomValueEstimate': np.random.uniform(50000, 500000, 1000)
    })
    print("Sample dataframe created for demonstration purposes.")


In [ ]:
# Display first few rows
print("First 5 rows:")
df.head()


In [ ]:
# Get data summary
summary = get_data_summary(df)
print("Data Summary:")
print(f"Shape: {summary['shape']}")
print(f"\nNumeric columns ({len(summary['numeric_columns'])}):")
print(summary['numeric_columns'])
print(f"\nCategorical columns ({len(summary['categorical_columns'])}):")
print(summary['categorical_columns'])
print(f"\nMemory usage: {summary['memory_usage_mb']:.2f} MB")


## 2. Data Quality Assessment


In [ ]:
# Check for missing values
missing_df = check_missing_values(df)
if len(missing_df) > 0:
    print("Missing Values Found:")
    print(missing_df)
else:
    print("✓ No missing values found in the dataset!")


In [ ]:
# Check data types
print("Data Types:")
print(df.dtypes)


## 3. Descriptive Statistics


In [ ]:
# Calculate descriptive statistics for key numeric columns
key_numeric_cols = ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 
                    'SumInsured', 'CalculatedPremiumPerTerm']
available_cols = [col for col in key_numeric_cols if col in df.columns]

if available_cols:
    desc_stats = calculate_descriptive_stats(df, available_cols)
    print("Descriptive Statistics for Key Financial Variables:")
    desc_stats
else:
    print("Key columns not found. Showing all numeric columns:")
    calculate_descriptive_stats(df)


## 4. Key Business Metrics


In [ ]:
# Calculate overall portfolio metrics
if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    overall_loss_ratio = calculate_loss_ratio(df['TotalClaims'], df['TotalPremium'])
    claim_frequency = calculate_claim_frequency(df)
    claim_severity = calculate_claim_severity(df)
    
    print("=== PORTFOLIO OVERVIEW ===")
    print(f"Overall Loss Ratio: {overall_loss_ratio:.4f} ({overall_loss_ratio*100:.2f}%)")
    print(f"Claim Frequency: {claim_frequency:.4f} ({claim_frequency*100:.2f}%)")
    print(f"Claim Severity: ZAR {claim_severity:,.2f}")
    print(f"\nTotal Premium: ZAR {df['TotalPremium'].sum():,.2f}")
    print(f"Total Claims: ZAR {df['TotalClaims'].sum():,.2f}")
    print(f"Total Margin: ZAR {(df['TotalPremium'].sum() - df['TotalClaims'].sum()):,.2f}")
else:
    print("Required columns (TotalPremium, TotalClaims) not found in dataset.")


In [ ]:
# Loss Ratio by Province
if 'Province' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    province_metrics = df.groupby('Province').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    province_metrics['LossRatio'] = province_metrics['TotalClaims'] / province_metrics['TotalPremium']
    province_metrics['ClaimFrequency'] = df.groupby('Province').apply(
        lambda x: calculate_claim_frequency(x)
    ).values
    province_metrics = province_metrics.sort_values('LossRatio', ascending=False)
    
    print("\n=== LOSS RATIO BY PROVINCE ===")
    print(province_metrics[['Province', 'PolicyID', 'TotalPremium', 'TotalClaims', 
                            'LossRatio', 'ClaimFrequency']].to_string(index=False))
else:
    print("Province column not found.")


In [ ]:
# Loss Ratio by Vehicle Type
if 'VehicleType' in df.columns:
    vehicle_metrics = df.groupby('VehicleType').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    vehicle_metrics['LossRatio'] = vehicle_metrics['TotalClaims'] / vehicle_metrics['TotalPremium']
    vehicle_metrics = vehicle_metrics.sort_values('LossRatio', ascending=False)
    
    print("\n=== LOSS RATIO BY VEHICLE TYPE ===")
    print(vehicle_metrics[['VehicleType', 'PolicyID', 'TotalPremium', 'TotalClaims', 
                           'LossRatio']].to_string(index=False))
else:
    print("VehicleType column not found.")


In [ ]:
# Loss Ratio by Gender
if 'Gender' in df.columns:
    gender_metrics = df.groupby('Gender').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    gender_metrics['LossRatio'] = gender_metrics['TotalClaims'] / gender_metrics['TotalPremium']
    gender_metrics['ClaimFrequency'] = df.groupby('Gender').apply(
        lambda x: calculate_claim_frequency(x)
    ).values
    
    print("\n=== LOSS RATIO BY GENDER ===")
    print(gender_metrics[['Gender', 'PolicyID', 'TotalPremium', 'TotalClaims', 
                          'LossRatio', 'ClaimFrequency']].to_string(index=False))
else:
    print("Gender column not found.")


## 5. Univariate Analysis - Distributions


In [ ]:
# Distribution of TotalPremium
if 'TotalPremium' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    plot_numeric_distribution(df, 'TotalPremium', bins=50, ax=axes[0])
    plot_numeric_distribution(df, 'TotalPremium', bins=50, log_scale=True, ax=axes[1])
    plt.tight_layout()
    plt.show()


In [ ]:
# Distribution of TotalClaims (only non-zero claims)
if 'TotalClaims' in df.columns:
    claims_nonzero = df[df['TotalClaims'] > 0]['TotalClaims']
    if len(claims_nonzero) > 0:
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        axes[0].hist(claims_nonzero, bins=50, edgecolor='black', alpha=0.7)
        axes[0].set_xlabel('TotalClaims')
        axes[0].set_ylabel('Frequency')
        axes[0].set_title('Distribution of TotalClaims (Non-zero only)')
        axes[0].grid(True, alpha=0.3)
        
        axes[1].hist(np.log1p(claims_nonzero), bins=50, edgecolor='black', alpha=0.7)
        axes[1].set_xlabel('Log(TotalClaims)')
        axes[1].set_ylabel('Frequency')
        axes[1].set_title('Distribution of TotalClaims (Log scale)')
        axes[1].grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()


In [ ]:
# Distribution of CustomValueEstimate
if 'CustomValueEstimate' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    plot_numeric_distribution(df, 'CustomValueEstimate', bins=50, ax=axes[0])
    plot_numeric_distribution(df, 'CustomValueEstimate', bins=50, log_scale=True, ax=axes[1])
    plt.tight_layout()
    plt.show()


In [ ]:
# Categorical distributions
if 'Province' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    plot_categorical_distribution(df, 'Province', top_n=15, ax=ax)
    plt.tight_layout()
    plt.show()


In [ ]:
# Vehicle Make distribution
if 'Make' in df.columns:
    fig, ax = plt.subplots(figsize=(12, 8))
    plot_categorical_distribution(df, 'Make', top_n=20, ax=ax)
    plt.tight_layout()
    plt.show()


## 6. Outlier Detection


In [ ]:
# Box plots for outlier detection - TotalClaims
if 'TotalClaims' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    plot_boxplot(df, 'TotalClaims', ax=ax)
    plt.tight_layout()
    plt.show()
    
    # Detect outliers
    outliers_info = detect_outliers_iqr(df, 'TotalClaims')
    print(f"\nOutliers in TotalClaims:")
    print(f"Count: {outliers_info['outlier_count']} ({outliers_info['outlier_percentage']:.2f}%)")
    print(f"Lower bound: {outliers_info['lower_bound']:.2f}")
    print(f"Upper bound: {outliers_info['upper_bound']:.2f}")


In [ ]:
# Box plots for outlier detection - CustomValueEstimate
if 'CustomValueEstimate' in df.columns:
    fig, ax = plt.subplots(figsize=(10, 6))
    plot_boxplot(df, 'CustomValueEstimate', ax=ax)
    plt.tight_layout()
    plt.show()
    
    # Detect outliers
    outliers_info = detect_outliers_iqr(df, 'CustomValueEstimate')
    print(f"\nOutliers in CustomValueEstimate:")
    print(f"Count: {outliers_info['outlier_count']} ({outliers_info['outlier_percentage']:.2f}%)")
    print(f"Lower bound: {outliers_info['lower_bound']:.2f}")
    print(f"Upper bound: {outliers_info['upper_bound']:.2f}")


In [ ]:
# Box plot by category - TotalClaims by Province
if 'Province' in df.columns and 'TotalClaims' in df.columns:
    fig, ax = plt.subplots(figsize=(14, 6))
    plot_boxplot(df, 'TotalClaims', by='Province', ax=ax)
    plt.tight_layout()
    plt.show()


## 7. Bivariate and Multivariate Analysis


In [ ]:
# Correlation matrix for numeric variables
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if len(numeric_cols) > 1:
    # Focus on key financial columns
    key_cols = [col for col in ['TotalPremium', 'TotalClaims', 'CustomValueEstimate', 
                                'SumInsured', 'CalculatedPremiumPerTerm'] 
                if col in numeric_cols]
    if len(key_cols) > 1:
        plot_correlation_matrix(df, key_cols)
        plt.show()
    else:
        plot_correlation_matrix(df, numeric_cols[:10])  # Limit to first 10 if too many
        plt.show()


In [ ]:
# Scatter plot: TotalPremium vs TotalClaims by PostalCode
if 'PostalCode' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    # Aggregate by PostalCode
    zipcode_stats = df.groupby('PostalCode').agg({
        'TotalPremium': 'sum',
        'TotalClaims': 'sum',
        'PolicyID': 'count'
    }).reset_index()
    
    fig, ax = plt.subplots(figsize=(12, 8))
    scatter = ax.scatter(zipcode_stats['TotalPremium'], zipcode_stats['TotalClaims'],
                        s=zipcode_stats['PolicyID']*2, alpha=0.6, 
                        c=zipcode_stats['PolicyID'], cmap='viridis',
                        edgecolors='black', linewidth=0.5)
    ax.set_xlabel('Total Premium (ZAR)', fontweight='bold')
    ax.set_ylabel('Total Claims (ZAR)', fontweight='bold')
    ax.set_title('TotalPremium vs TotalClaims by PostalCode\n(Bubble size = Number of Policies)', 
                fontweight='bold')
    ax.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax, label='Number of Policies')
    plt.tight_layout()
    plt.show()
    
    # Calculate correlation
    correlation = zipcode_stats['TotalPremium'].corr(zipcode_stats['TotalClaims'])
    print(f"\nCorrelation between TotalPremium and TotalClaims (by PostalCode): {correlation:.4f}")


## 8. Temporal Trends Analysis


In [ ]:
# Monthly trends in Premium and Claims
if 'TransactionMonth' in df.columns:
    df['TransactionMonth'] = pd.to_datetime(df['TransactionMonth'], errors='coerce')
    
    if 'TotalPremium' in df.columns:
        plot_temporal_trends(df, 'TransactionMonth', 'TotalPremium', agg_func='sum')
        plt.show()
    
    if 'TotalClaims' in df.columns:
        plot_temporal_trends(df, 'TransactionMonth', 'TotalClaims', agg_func='sum')
        plt.show()
    
    # Monthly loss ratio trend
    if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
        monthly_stats = df.groupby(df['TransactionMonth'].dt.to_period('M')).agg({
            'TotalPremium': 'sum',
            'TotalClaims': 'sum',
            'PolicyID': 'count'
        }).reset_index()
        monthly_stats['LossRatio'] = monthly_stats['TotalClaims'] / monthly_stats['TotalPremium']
        
        fig, ax = plt.subplots(figsize=(14, 6))
        ax.plot(monthly_stats['TransactionMonth'].astype(str), monthly_stats['LossRatio'], 
               marker='o', linewidth=2, markersize=8)
        ax.axhline(y=1.0, color='r', linestyle='--', linewidth=2, label='Break-even')
        ax.set_xlabel('Month')
        ax.set_ylabel('Loss Ratio')
        ax.set_title('Monthly Loss Ratio Trend', fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
        
        # Claim frequency over time
        monthly_claim_freq = df.groupby(df['TransactionMonth'].dt.to_period('M')).apply(
            lambda x: calculate_claim_frequency(x)
        ).reset_index()
        monthly_claim_freq.columns = ['TransactionMonth', 'ClaimFrequency']
        
        fig, ax = plt.subplots(figsize=(14, 6))
        ax.plot(monthly_claim_freq['TransactionMonth'].astype(str), 
               monthly_claim_freq['ClaimFrequency'], marker='o', linewidth=2, markersize=8)
        ax.set_xlabel('Month')
        ax.set_ylabel('Claim Frequency')
        ax.set_title('Monthly Claim Frequency Trend', fontweight='bold')
        ax.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
else:
    print("TransactionMonth column not found.")


In [ ]:
# Loss Ratio by Province
if 'Province' in df.columns:
    plot_loss_ratio_by_category(df, 'Province', top_n=15)
    plt.savefig('../figures/loss_ratio_by_province.png', dpi=300, bbox_inches='tight')
    plt.show()


In [ ]:
# Top and bottom vehicle makes by claim amount
if 'Make' in df.columns and 'TotalClaims' in df.columns:
    make_claims = df.groupby('Make').agg({
        'TotalClaims': ['sum', 'mean', 'count']
    }).reset_index()
    make_claims.columns = ['Make', 'TotalClaims_Sum', 'AvgClaims', 'PolicyCount']
    make_claims = make_claims[make_claims['PolicyCount'] >= 10]  # Filter for sufficient sample
    
    print("=== TOP 10 VEHICLE MAKES BY TOTAL CLAIMS ===")
    print(make_claims.nlargest(10, 'TotalClaims_Sum')[['Make', 'TotalClaims_Sum', 'AvgClaims', 'PolicyCount']].to_string(index=False))
    
    print("\n=== BOTTOM 10 VEHICLE MAKES BY TOTAL CLAIMS ===")
    print(make_claims.nsmallest(10, 'TotalClaims_Sum')[['Make', 'TotalClaims_Sum', 'AvgClaims', 'PolicyCount']].to_string(index=False))


## 10. Creative Visualizations - Key Insights

### Visualization 1: Risk-Return Analysis by Province


In [ ]:
# Create insightful visualization 1
if 'Province' in df.columns and 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    create_insightful_visualization_1(df, save_path='../figures/risk_return_by_province.png')
    plt.show()


### Visualization 2: Temporal Analysis with Trends


In [ ]:
# Create insightful visualization 2
if 'TransactionMonth' in df.columns:
    create_insightful_visualization_2(df, save_path='../figures/temporal_trends_analysis.png')
    plt.show()


### Visualization 3: Vehicle Risk Profile Analysis


In [ ]:
# Create insightful visualization 3
if 'Make' in df.columns and 'VehicleType' in df.columns:
    create_insightful_visualization_3(df, save_path='../figures/vehicle_risk_profile.png')
    plt.show()


## 11. Summary of Key Findings

### Key Insights from EDA:

1. **Portfolio Overview**: [Summarize overall loss ratio, claim frequency, and severity]

2. **Geographic Patterns**: [Summarize findings by Province and PostalCode]

3. **Vehicle Characteristics**: [Summarize findings by Make, Model, VehicleType]

4. **Demographic Patterns**: [Summarize findings by Gender]

5. **Temporal Trends**: [Summarize monthly trends and seasonality]

6. **Risk Factors**: [Identify key risk drivers]

7. **Outliers**: [Summarize outlier findings]

8. **Data Quality**: [Summarize data quality issues]


In [ ]:
# Generate summary statistics for report
print("="*80)
print("EDA SUMMARY REPORT")
print("="*80)

if 'TotalPremium' in df.columns and 'TotalClaims' in df.columns:
    print(f"\n1. Portfolio Metrics:")
    print(f"   - Total Policies: {len(df):,}")
    print(f"   - Overall Loss Ratio: {calculate_loss_ratio(df['TotalClaims'], df['TotalPremium']):.4f}")
    print(f"   - Claim Frequency: {calculate_claim_frequency(df):.4f}")
    print(f"   - Claim Severity: ZAR {calculate_claim_severity(df):,.2f}")

if 'Province' in df.columns:
    print(f"\n2. Geographic Distribution:")
    print(f"   - Number of Provinces: {df['Province'].nunique()}")
    print(f"   - Top Province by Policy Count: {df['Province'].value_counts().index[0]}")

if 'Make' in df.columns:
    print(f"\n3. Vehicle Distribution:")
    print(f"   - Number of Makes: {df['Make'].nunique()}")
    print(f"   - Top Make: {df['Make'].value_counts().index[0]}")

print("\n" + "="*80)
print("EDA Complete! Review visualizations and findings above.")
print("="*80)
